In [1]:
  # Install required packages
!pip install pandas neo4j llama-index python-dotenv

In [14]:
import pandas as pd
import numpy as np
from neo4j import GraphDatabase
from datetime import datetime
import os
from dotenv import load_dotenv

# Neo4j connection settings
NEO4J_URI = "bolt://localhost:7687"  
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password123"  

class HealthcareGraphBuilder:
  def __init__(self, uri, user, password):
      self.driver = GraphDatabase.driver(uri, auth=(user, password))

  def close(self):
      self.driver.close()

  def verify_connection(self):
      try:
          with self.driver.session() as session:
              result = session.run("RETURN 1")
              print("Successfully connected to Neo4j!")
              return True
      except Exception as e:
          print(f"Failed to connect to Neo4j: {str(e)}")
          return False

In [3]:
def create_constraints(self):
  constraints = [
      "CREATE CONSTRAINT patient_id IF NOT EXISTS FOR (p:Patient) REQUIRE p.patient_id IS UNIQUE",
      "CREATE CONSTRAINT diagnosis_name IF NOT EXISTS FOR (d:Diagnosis) REQUIRE d.name IS UNIQUE",
      "CREATE CONSTRAINT location_postcode IF NOT EXISTS FOR (l:Location) REQUIRE l.postcode IS UNIQUE",
      "CREATE CONSTRAINT visit_id IF NOT EXISTS FOR (v:Visit) REQUIRE v.visit_id IS UNIQUE"
  ]

  with self.driver.session() as session:
      for constraint in constraints:
          try:
              session.run(constraint)
          except Exception as e:
              print(f"Error creating constraint: {str(e)}")

def clear_database(self):
  with self.driver.session() as session:
      session.run("MATCH (n) DETACH DELETE n")
      print("Database cleared successfully")

In [15]:
def create_patient(self, tx, row):
  query = """
  MERGE (p:Patient {patient_id: $patient_id})
  SET p.age = $age,
      p.gender = $gender,
      p.has_insurance = $has_insurance
  RETURN p
  """
  return tx.run(query,
               patient_id=row['patient_ID'],
               age=row['age'],
               gender=row['gender'],
               has_insurance=row['has_insurance'])

def create_visit(self, tx, row):
  visit_id = f"{row['patient_ID']}_{row['Visit date']}"
  query = """
  MATCH (p:Patient {patient_id: $patient_id})
  CREATE (v:Visit {
      visit_id: $visit_id,
      visit_date: date($visit_date),
      total_cost: $total_cost,
      registration_time: $registration,
      nursing_time: $nursing,
      consultation_time: $consultation,
      laboratory_time: $laboratory,
      pharmacy_time: $pharmacy
  })
  CREATE (p)-[:PATIENT_VISIT]->(v)
  RETURN v
  """
  return tx.run(query,
               patient_id=row['patient_ID'],
               visit_id=visit_id,
               visit_date=row['Visit date'],
               total_cost=row['total_cost'],
               registration=row['registration'],
               nursing=row['nursing'],
               consultation=row['consultation'],
               laboratory=row['laboratory'],
               pharmacy=row['pharmacy'])

def create_diagnosis(self, tx, row, visit_id):
  query = """
  MATCH (v:Visit {visit_id: $visit_id})
  MERGE (d:Diagnosis {name: $diagnosis})
  CREATE (v)-[:VISIT_DIAGNOSIS]->(d)
  """
  return tx.run(query,
               visit_id=visit_id,
               diagnosis=row['diagnosis'])

def create_location(self, tx, row):
  query = """
  MATCH (p:Patient {patient_id: $patient_id})
  MERGE (l:Location {postcode: $postcode})
  CREATE (p)-[:PATIENT_LOCATION]->(l)
  """
  return tx.run(query,
               patient_id=row['patient_ID'],
               postcode=row['postcode'])

In [5]:
def process_data(self, df):
  print(f"Processing {len(df)} records...")

  with self.driver.session() as session:
      # Clear existing data
      self.clear_database()

      # Create constraints
      self.create_constraints()

      # Process each row
      for index, row in df.iterrows():
          visit_id = f"{row['patient_ID']}_{row['Visit date']}"

          # Create nodes and relationships
          session.execute_write(self.create_patient, row)
          session.execute_write(self.create_visit, row)
          session.execute_write(self.create_diagnosis, row, visit_id)
          session.execute_write(self.create_location, row)

          if index % 100 == 0:
              print(f"Processed {index + 1} records...")

  print("Data import completed!")

In [12]:
print(df.head())  # Check the first few rows
print(df.columns)  # Verify column names


   Visit_Date  Patient_ID  Age  Gender                          Diagnosis  \
0  2020-05-06      688923   68  Female                           Diabetes   
1  2018-08-04      886361   62  Female            Urinary Tract Infection   
2  2021-04-10      464823   70  Female  Upper Respiratory Tract Infection   
3  2021-10-01      655214    8  Female  Upper Respiratory Tract Infection   
4  2018-04-30      454666   24    Male                            Malaria   

   Has_Insurance  Postcode  Total_Cost  Registration  Nursing  Laboratory  \
0           True     20006      2274.0            33      105         101   
1          False     20005      3430.0            46       75          84   
2           True     10003      1836.0           107       81           5   
3          False     10006      3250.0            66       31         101   
4           True     10006      2262.0            68       55          60   

   Consultation  Pharmacy  
0            28       112  
1           102   

In [19]:
import pandas as pd
from neo4j import GraphDatabase

class HealthcareGraphBuilder:
  def __init__(self, uri, user, password):
      self.driver = GraphDatabase.driver(uri, auth=(user, password))

  def close(self):
      self.driver.close()

  def verify_connection(self):
      try:
          with self.driver.session() as session:
              result = session.run("RETURN 1")
              print("Successfully connected to Neo4j!")
              return True
      except Exception as e:
          print(f"Failed to connect to Neo4j: {str(e)}")
          return False

  def create_patient(self, tx, row):
      query = """
      MERGE (p:Patient {patient_id: $patient_id})
      SET p.age = $age,
          p.gender = $gender,
          p.has_insurance = $has_insurance
      RETURN p
      """
      return tx.run(query,
                   patient_id=row['Patient_ID'],  
                   age=row['Age'],               
                   gender=row['Gender'],         
                   has_insurance=row['Has_Insurance'])  

  def create_visit(self, tx, row):
      visit_id = f"{row['Patient_ID']}_{row['Visit_Date']}"  
      query = """
      MATCH (p:Patient {patient_id: $patient_id})
      CREATE (v:Visit {
          visit_id: $visit_id,
          visit_date: date($visit_date),
          total_cost: $total_cost,
          registration_time: $registration,
          nursing_time: $nursing,
          consultation_time: $consultation,
          laboratory_time: $laboratory,
          pharmacy_time: $pharmacy
      })
      CREATE (p)-[:PATIENT_VISIT]->(v)
      RETURN v
      """
      return tx.run(query,
                   patient_id=row['Patient_ID'],     
                   visit_id=visit_id,
                   visit_date=row['Visit_Date'],     
                   total_cost=row['Total_Cost'],    
                   registration=row['Registration'], 
                   nursing=row['Nursing'],           
                   consultation=row['Consultation'],  
                   laboratory=row['Laboratory'],      
                   pharmacy=row['Pharmacy'])         

  def create_diagnosis(self, tx, row, visit_id):
      query = """
      MATCH (v:Visit {visit_id: $visit_id})
      MERGE (d:Diagnosis {name: $diagnosis})
      CREATE (v)-[:VISIT_DIAGNOSIS]->(d)
      """
      return tx.run(query,
                   visit_id=visit_id,
                   diagnosis=row['Diagnosis'])  

  def create_location(self, tx, row):
      query = """
      MATCH (p:Patient {patient_id: $patient_id})
      MERGE (l:Location {postcode: $postcode})
      CREATE (p)-[:PATIENT_LOCATION]->(l)
      """
      return tx.run(query,
                   patient_id=row['Patient_ID'],  
                   postcode=row['Postcode'])     

  def process_data(self, df):
      print(f"Processing {len(df)} records...")

      with self.driver.session() as session:
          # Clear existing data
          self.clear_database()

          # Create constraints
          self.create_constraints()

          # Process each row
          for index, row in df.iterrows():
              visit_id = f"{row['Patient_ID']}_{row['Visit_Date']}" 

              # Create nodes and relationships
              session.execute_write(self.create_patient, row)
              session.execute_write(self.create_visit, row)
              session.execute_write(self.create_diagnosis, row, visit_id)
              session.execute_write(self.create_location, row)

              if index % 100 == 0:
                  print(f"Processed {index + 1} records...")

      print("Data import completed!")

  def clear_database(self):
      with self.driver.session() as session:
          session.run("MATCH (n) DETACH DELETE n")
          print("Database cleared successfully")

  def create_constraints(self):
      constraints = [
          "CREATE CONSTRAINT patient_id IF NOT EXISTS FOR (p:Patient) REQUIRE p.patient_id IS UNIQUE",
          "CREATE CONSTRAINT diagnosis_name IF NOT EXISTS FOR (d:Diagnosis) REQUIRE d.name IS UNIQUE",
          "CREATE CONSTRAINT location_postcode IF NOT EXISTS FOR (l:Location) REQUIRE l.postcode IS UNIQUE",
          "CREATE CONSTRAINT visit_id IF NOT EXISTS FOR (v:Visit) REQUIRE v.visit_id IS UNIQUE"
      ]

      with self.driver.session() as session:
          for constraint in constraints:
              try:
                  session.run(constraint)
              except Exception as e:
                  print(f"Error creating constraint: {str(e)}")

# Neo4j connection settings
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password123"  

# Read the CSV file
df = pd.read_csv(r"C:\Users\pc\Downloads\Demo Hospital Outpatient Data_NHC.csv")

# Initialize graph builder
graph_builder = HealthcareGraphBuilder(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)

try:
  # Verify connection
  if graph_builder.verify_connection():
      # Process data
      graph_builder.process_data(df)
finally:
  graph_builder.close()

Successfully connected to Neo4j!
Processing 1000000 records...
Database cleared successfully
Processed 1 records...
Processed 101 records...
Processed 201 records...
Processed 301 records...
Processed 401 records...
Processed 501 records...
Processed 601 records...
Processed 701 records...
Processed 801 records...
Processed 901 records...
Processed 1001 records...
Processed 1101 records...
Processed 1201 records...
Processed 1301 records...
Processed 1401 records...
Processed 1501 records...
Processed 1601 records...
Processed 1701 records...
Processed 1801 records...
Processed 1901 records...
Processed 2001 records...
Processed 2101 records...
Processed 2201 records...
Processed 2301 records...
Processed 2401 records...
Processed 2501 records...
Processed 2601 records...
Processed 2701 records...
Processed 2801 records...
Processed 2901 records...
Processed 3001 records...
Processed 3101 records...
Processed 3201 records...
Processed 3301 records...
Processed 3401 records...
Processed

Processed 30501 records...
Processed 30601 records...
Processed 30701 records...
Processed 30801 records...
Processed 30901 records...
Processed 31001 records...
Processed 31101 records...
Processed 31201 records...
Processed 31301 records...
Processed 31401 records...
Processed 31501 records...
Processed 31601 records...
Processed 31701 records...
Processed 31801 records...
Processed 31901 records...
Processed 32001 records...
Processed 32101 records...
Processed 32201 records...
Processed 32301 records...
Processed 32401 records...
Processed 32501 records...
Processed 32601 records...
Processed 32701 records...
Processed 32801 records...
Processed 32901 records...
Processed 33001 records...
Processed 33101 records...
Processed 33201 records...
Processed 33301 records...
Processed 33401 records...
Processed 33501 records...
Processed 33601 records...
Processed 33701 records...
Processed 33801 records...
Processed 33901 records...
Processed 34001 records...
Processed 34101 records...
P

ConstraintError: {code: Neo.ClientError.Schema.ConstraintValidationFailed} {message: Node(31281) already exists with label `Visit` and property `visit_id` = '430954_2018-01-17'}

In [20]:
def create_visit(self, tx, row):
  # Add a random suffix to make visit_id unique
  import uuid
  visit_id = f"{row['Patient_ID']}_{row['Visit_Date']}_{str(uuid.uuid4())[:8]}"

  query = """
  MERGE (p:Patient {patient_id: $patient_id})
  CREATE (v:Visit {
      visit_id: $visit_id,
      visit_date: date($visit_date),
      total_cost: $total_cost,
      registration_time: $registration,
      nursing_time: $nursing,
      consultation_time: $consultation,
      laboratory_time: $laboratory,
      pharmacy_time: $pharmacy
  })
  CREATE (p)-[:PATIENT_VISIT]->(v)
  RETURN v
  """
  return tx.run(query,
               patient_id=row['Patient_ID'],
               visit_id=visit_id,
               visit_date=row['Visit_Date'],
               total_cost=row['Total_Cost'],
               registration=row['Registration'],
               nursing=row['Nursing'],
               consultation=row['Consultation'],
               laboratory=row['Laboratory'],
               pharmacy=row['Pharmacy'])

In [22]:
import pandas as pd
from neo4j import GraphDatabase
import uuid

class HealthcareGraphBuilder:
  def __init__(self, uri, user, password):
      self.driver = GraphDatabase.driver(uri, auth=(user, password))

  def close(self):
      self.driver.close()

  def verify_connection(self):
      try:
          with self.driver.session() as session:
              result = session.run("RETURN 1")
              print("Successfully connected to Neo4j!")
              return True
      except Exception as e:
          print(f"Failed to connect to Neo4j: {str(e)}")
          return False

  def create_patient(self, tx, row):
      query = """
      MERGE (p:Patient {patient_id: $patient_id})
      SET p.age = $age,
          p.gender = $gender,
          p.has_insurance = $has_insurance
      RETURN p
      """
      return tx.run(query,
                   patient_id=row['Patient_ID'],
                   age=row['Age'],
                   gender=row['Gender'],
                   has_insurance=row['Has_Insurance'])

  def create_visit(self, tx, row):
      # Add a random suffix to make visit_id unique
      visit_id = f"{row['Patient_ID']}_{row['Visit_Date']}_{str(uuid.uuid4())[:8]}"

      query = """
      MATCH (p:Patient {patient_id: $patient_id})
      CREATE (v:Visit {
          visit_id: $visit_id,
          visit_date: date($visit_date),
          total_cost: $total_cost,
          registration_time: $registration,
          nursing_time: $nursing,
          consultation_time: $consultation,
          laboratory_time: $laboratory,
          pharmacy_time: $pharmacy
      })
      CREATE (p)-[:PATIENT_VISIT]->(v)
      RETURN v
      """
      return tx.run(query,
                   patient_id=row['Patient_ID'],
                   visit_id=visit_id,
                   visit_date=row['Visit_Date'],
                   total_cost=row['Total_Cost'],
                   registration=row['Registration'],
                   nursing=row['Nursing'],
                   consultation=row['Consultation'],
                   laboratory=row['Laboratory'],
                   pharmacy=row['Pharmacy'])

  def create_diagnosis(self, tx, row, visit_id):
      query = """
      MATCH (v:Visit {visit_id: $visit_id})
      MERGE (d:Diagnosis {name: $diagnosis})
      CREATE (v)-[:VISIT_DIAGNOSIS]->(d)
      """
      return tx.run(query,
                   visit_id=visit_id,
                   diagnosis=row['Diagnosis'])

  def create_location(self, tx, row):
      query = """
      MATCH (p:Patient {patient_id: $patient_id})
      MERGE (l:Location {postcode: $postcode})
      CREATE (p)-[:PATIENT_LOCATION]->(l)
      """
      return tx.run(query,
                   patient_id=row['Patient_ID'],
                   postcode=row['Postcode'])

  def process_data(self, df):
      print(f"Processing {len(df)} records...")

      with self.driver.session() as session:
          # Clear existing data
          self.clear_database()

          # Create constraints
          self.create_constraints()

          # Process each row
          for index, row in df.iterrows():
              visit_id = f"{row['Patient_ID']}_{row['Visit_Date']}_{str(uuid.uuid4())[:8]}"

              # Create nodes and relationships
              session.execute_write(self.create_patient, row)
              session.execute_write(self.create_visit, row)
              session.execute_write(self.create_diagnosis, row, visit_id)
              session.execute_write(self.create_location, row)

              if index % 100 == 0:
                  print(f"Processed {index + 1} records...")

      print("Data import completed!")

  def clear_database(self):
      with self.driver.session() as session:
          session.run("MATCH (n) DETACH DELETE n")
          print("Database cleared successfully")

  def create_constraints(self):
      constraints = [
          "CREATE CONSTRAINT patient_id IF NOT EXISTS FOR (p:Patient) REQUIRE p.patient_id IS UNIQUE",
          "CREATE CONSTRAINT diagnosis_name IF NOT EXISTS FOR (d:Diagnosis) REQUIRE d.name IS UNIQUE",
          "CREATE CONSTRAINT location_postcode IF NOT EXISTS FOR (l:Location) REQUIRE l.postcode IS UNIQUE",
          "CREATE CONSTRAINT visit_id IF NOT EXISTS FOR (v:Visit) REQUIRE v.visit_id IS UNIQUE"
      ]

      with self.driver.session() as session:
          for constraint in constraints:
              try:
                  session.run(constraint)
              except Exception as e:
                  print(f"Error creating constraint: {str(e)}")

# Neo4j connection settings
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password123"  # Replace with your actual password

# Read the CSV file
df = pd.read_csv(r"C:\Users\pc\Downloads\Demo Hospital Outpatient Data_NHC.csv")

# Initialize graph builder
graph_builder = HealthcareGraphBuilder(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)

try:
  # Verify connection
  if graph_builder.verify_connection():
      # Process data
      graph_builder.process_data(df)
finally:
  graph_builder.close()

Successfully connected to Neo4j!
Processing 1000000 records...
Database cleared successfully
Processed 1 records...
Processed 101 records...
Processed 201 records...
Processed 301 records...
Processed 401 records...
Processed 501 records...
Processed 601 records...
Processed 701 records...
Processed 801 records...
Processed 901 records...
Processed 1001 records...
Processed 1101 records...
Processed 1201 records...
Processed 1301 records...
Processed 1401 records...
Processed 1501 records...
Processed 1601 records...
Processed 1701 records...
Processed 1801 records...
Processed 1901 records...
Processed 2001 records...
Processed 2101 records...
Processed 2201 records...
Processed 2301 records...
Processed 2401 records...
Processed 2501 records...
Processed 2601 records...
Processed 2701 records...
Processed 2801 records...
Processed 2901 records...
Processed 3001 records...
Processed 3101 records...
Processed 3201 records...
Processed 3301 records...
Processed 3401 records...
Processed

Processed 30501 records...
Processed 30601 records...
Processed 30701 records...
Processed 30801 records...
Processed 30901 records...
Processed 31001 records...
Processed 31101 records...
Processed 31201 records...
Processed 31301 records...
Processed 31401 records...
Processed 31501 records...
Processed 31601 records...
Processed 31701 records...
Processed 31801 records...
Processed 31901 records...
Processed 32001 records...
Processed 32101 records...
Processed 32201 records...
Processed 32301 records...
Processed 32401 records...
Processed 32501 records...
Processed 32601 records...
Processed 32701 records...
Processed 32801 records...
Processed 32901 records...
Processed 33001 records...
Processed 33101 records...
Processed 33201 records...
Processed 33301 records...
Processed 33401 records...
Processed 33501 records...
Processed 33601 records...
Processed 33701 records...
Processed 33801 records...
Processed 33901 records...
Processed 34001 records...
Processed 34101 records...
P

Processed 60901 records...
Processed 61001 records...
Processed 61101 records...
Processed 61201 records...
Processed 61301 records...
Processed 61401 records...
Processed 61501 records...
Processed 61601 records...
Processed 61701 records...
Processed 61801 records...
Processed 61901 records...
Processed 62001 records...
Processed 62101 records...
Processed 62201 records...
Processed 62301 records...
Processed 62401 records...
Processed 62501 records...
Processed 62601 records...
Processed 62701 records...
Processed 62801 records...
Processed 62901 records...
Processed 63001 records...
Processed 63101 records...
Processed 63201 records...
Processed 63301 records...
Processed 63401 records...
Processed 63501 records...
Processed 63601 records...
Processed 63701 records...
Processed 63801 records...
Processed 63901 records...
Processed 64001 records...
Processed 64101 records...
Processed 64201 records...
Processed 64301 records...
Processed 64401 records...
Processed 64501 records...
P

Processed 91301 records...
Processed 91401 records...
Processed 91501 records...
Processed 91601 records...
Processed 91701 records...
Processed 91801 records...
Processed 91901 records...
Processed 92001 records...
Processed 92101 records...
Processed 92201 records...
Processed 92301 records...
Processed 92401 records...
Processed 92501 records...
Processed 92601 records...
Processed 92701 records...
Processed 92801 records...
Processed 92901 records...
Processed 93001 records...
Processed 93101 records...
Processed 93201 records...
Processed 93301 records...
Processed 93401 records...
Processed 93501 records...
Processed 93601 records...
Processed 93701 records...
Processed 93801 records...
Processed 93901 records...
Processed 94001 records...
Processed 94101 records...
Processed 94201 records...
Processed 94301 records...
Processed 94401 records...
Processed 94501 records...
Processed 94601 records...
Processed 94701 records...
Processed 94801 records...
Processed 94901 records...
P

Processed 120901 records...
Processed 121001 records...
Processed 121101 records...
Processed 121201 records...
Processed 121301 records...
Processed 121401 records...
Processed 121501 records...
Processed 121601 records...
Processed 121701 records...
Processed 121801 records...
Processed 121901 records...
Processed 122001 records...
Processed 122101 records...
Processed 122201 records...
Processed 122301 records...
Processed 122401 records...
Processed 122501 records...
Processed 122601 records...
Processed 122701 records...
Processed 122801 records...
Processed 122901 records...
Processed 123001 records...
Processed 123101 records...
Processed 123201 records...
Processed 123301 records...
Processed 123401 records...
Processed 123501 records...
Processed 123601 records...
Processed 123701 records...
Processed 123801 records...
Processed 123901 records...
Processed 124001 records...
Processed 124101 records...
Processed 124201 records...
Processed 124301 records...
Processed 124401 rec

...

Processed 882701 records...
Processed 882801 records...
Processed 882901 records...
Processed 883001 records...
Processed 883101 records...
Processed 883201 records...
Processed 883301 records...
Processed 883401 records...
Processed 883501 records...
Processed 883601 records...
Processed 883701 records...
Processed 883801 records...
Processed 883901 records...
Processed 884001 records...
Processed 884101 records...
Processed 884201 records...
Processed 884301 records...
Processed 884401 records...
Processed 884501 records...
Processed 884601 records...
Processed 884701 records...
Processed 884801 records...
Processed 884901 records...
Processed 885001 records...
Processed 885101 records...
Processed 885201 records...
Processed 885301 records...
Processed 885401 records...
Processed 885501 records...
Processed 885601 records...
Processed 885701 records...
Processed 885801 records...
Processed 885901 records...
Processed 886001 records...
Processed 886101 records...
Processed 886201 rec

Processed 912001 records...
Processed 912101 records...
Processed 912201 records...
Processed 912301 records...
Processed 912401 records...
Processed 912501 records...
Processed 912601 records...
Processed 912701 records...
Processed 912801 records...
Processed 912901 records...
Processed 913001 records...
Processed 913101 records...
Processed 913201 records...
Processed 913301 records...
Processed 913401 records...
Processed 913501 records...
Processed 913601 records...
Processed 913701 records...
Processed 913801 records...
Processed 913901 records...
Processed 914001 records...
Processed 914101 records...
Processed 914201 records...
Processed 914301 records...
Processed 914401 records...
Processed 914501 records...
Processed 914601 records...
Processed 914701 records...
Processed 914801 records...
Processed 914901 records...
Processed 915001 records...
Processed 915101 records...
Processed 915201 records...
Processed 915301 records...
Processed 915401 records...
Processed 915501 rec

Processed 941301 records...
Processed 941401 records...
Processed 941501 records...
Processed 941601 records...
Processed 941701 records...
Processed 941801 records...
Processed 941901 records...
Processed 942001 records...
Processed 942101 records...
Processed 942201 records...
Processed 942301 records...
Processed 942401 records...
Processed 942501 records...
Processed 942601 records...
Processed 942701 records...
Processed 942801 records...
Processed 942901 records...
Processed 943001 records...
Processed 943101 records...
Processed 943201 records...
Processed 943301 records...
Processed 943401 records...
Processed 943501 records...
Processed 943601 records...
Processed 943701 records...
Processed 943801 records...
Processed 943901 records...
Processed 944001 records...
Processed 944101 records...
Processed 944201 records...
Processed 944301 records...
Processed 944401 records...
Processed 944501 records...
Processed 944601 records...
Processed 944701 records...
Processed 944801 rec

Processed 970601 records...
Processed 970701 records...
Processed 970801 records...
Processed 970901 records...
Processed 971001 records...
Processed 971101 records...
Processed 971201 records...
Processed 971301 records...
Processed 971401 records...
Processed 971501 records...
Processed 971601 records...
Processed 971701 records...
Processed 971801 records...
Processed 971901 records...
Processed 972001 records...
Processed 972101 records...
Processed 972201 records...
Processed 972301 records...
Processed 972401 records...
Processed 972501 records...
Processed 972601 records...
Processed 972701 records...
Processed 972801 records...
Processed 972901 records...
Processed 973001 records...
Processed 973101 records...
Processed 973201 records...
Processed 973301 records...
Processed 973401 records...
Processed 973501 records...
Processed 973601 records...
Processed 973701 records...
Processed 973801 records...
Processed 973901 records...
Processed 974001 records...
Processed 974101 rec

Processed 999901 records...
Data import completed!
